This ipynb file is to serve as production/test for the different sections of the algorithm/function module for the project in a step by step process before pushing to the final version

In [2]:
# To check the current python path
import sys
print(sys.executable)

/bin/python3


In [3]:
# Yahoo Finance API to pull the data
import yfinance as yf
# Pandas and Numpy for data maniupulation 
import pandas as pd
import numpy as np
# Matflotlib and Ploty for visualizations (Testing purposes)
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

Data preprocessing and api pull (to take not that Aamir and Dalton is doing this)

In [4]:
# Example tickers (one per sector)
tickers = ['AAPL', 'JPM', 'PFE', 'PG', 'XOM']
sectors = {
    'AAPL': 'Technology',
    'JPM': 'Finance',
    'PFE': 'Healthcare',
    'PG': 'Consumer Goods',
    'XOM': 'Energy'
}

# Function to fetch and flatten stock data
def fetch_stock_data(tickers, start_date='2023-01-01', end_date='2025-01-01'):
    df = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', progress=False)
    flat_data = []
    for ticker in tickers:
        temp = df[ticker].reset_index()
        temp['Ticker'] = ticker
        temp['Sector'] = sectors[ticker]  # Add sector info
        flat_data.append(temp)
    combined_df = pd.concat(flat_data, ignore_index=True)
    return combined_df

# Fetch data
df = fetch_stock_data(tickers)
df.head()


/tmp/ipykernel_3881/572029278.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', progress=False)


Price,Date,Open,High,Low,Close,Volume,Ticker,Sector
0,2023-01-03,128.468194,129.079567,122.443165,123.330650,112117500,AAPL,Technology
1,2023-01-04,125.125343,126.870731,123.340517,124.602715,89113600,AAPL,Technology
2,2023-01-05,125.361991,125.993089,123.024955,123.281334,80962700,AAPL,Technology
3,2023-01-06,124.257586,128.478056,123.153159,127.817375,87754700,AAPL,Technology
4,2023-01-09,128.655553,131.554669,128.083618,128.339996,70790800,AAPL,Technology


In [5]:
print(df.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker', 'Sector'], dtype='object', name='Price')


Start of algos

In [6]:
def calculate_sma(df, window=5):
    df_list = []
    for ticker, group in df.groupby('Ticker'):
        group = group.sort_values('Date')
        group['SMA_' + str(window)] = group['Close'].rolling(window).mean()
        df_list.append(group)
    return pd.concat(df_list, ignore_index=True)

df = calculate_sma(df, window=5)
df.head()

Price,Date,Open,High,Low,Close,Volume,Ticker,Sector,SMA_5
0,2023-01-03,128.468194,129.079567,122.443165,123.330650,112117500,AAPL,Technology,NaN
1,2023-01-04,125.125343,126.870731,123.340517,124.602715,89113600,AAPL,Technology,NaN
2,2023-01-05,125.361991,125.993089,123.024955,123.281334,80962700,AAPL,Technology,NaN
3,2023-01-06,124.257586,128.478056,123.153159,127.817375,87754700,AAPL,Technology,NaN
4,2023-01-09,128.655553,131.554669,128.083618,128.339996,70790800,AAPL,Technology,125.474414


In [7]:
df = calculate_sma(df, window=20)
df.head()

Price,Date,Open,High,Low,Close,Volume,Ticker,Sector,SMA_5,SMA_20
0,2023-01-03,128.468194,129.079567,122.443165,123.330650,112117500,AAPL,Technology,NaN,NaN
1,2023-01-04,125.125343,126.870731,123.340517,124.602715,89113600,AAPL,Technology,NaN,NaN
2,2023-01-05,125.361991,125.993089,123.024955,123.281334,80962700,AAPL,Technology,NaN,NaN
3,2023-01-06,124.257586,128.478056,123.153159,127.817375,87754700,AAPL,Technology,NaN,NaN
4,2023-01-09,128.655553,131.554669,128.083618,128.339996,70790800,AAPL,Technology,125.474414,NaN


In [8]:
def calculate_daily_returns(df):
    df_list = []
    for ticker, group in df.groupby('Ticker'):
        group = group.sort_values('Date')
        group['Daily_Return'] = group['Close'].pct_change()
        df_list.append(group)
    return pd.concat(df_list, ignore_index=True)

df = calculate_daily_returns(df)
df.head()


Price,Date,Open,High,Low,Close,Volume,Ticker,Sector,SMA_5,SMA_20,Daily_Return
0,2023-01-03,128.468194,129.079567,122.443165,123.330650,112117500,AAPL,Technology,NaN,NaN,NaN
1,2023-01-04,125.125343,126.870731,123.340517,124.602715,89113600,AAPL,Technology,NaN,NaN,0.010314
2,2023-01-05,125.361991,125.993089,123.024955,123.281334,80962700,AAPL,Technology,NaN,NaN,-0.010605
3,2023-01-06,124.257586,128.478056,123.153159,127.817375,87754700,AAPL,Technology,NaN,NaN,0.036794
4,2023-01-09,128.655553,131.554669,128.083618,128.339996,70790800,AAPL,Technology,125.474414,NaN,0.004089


In [9]:
def detect_trends(df):
    df_list = []
    for ticker, group in df.groupby('Ticker'):
        group = group.sort_values('Date')
        run = [0]*len(group)
        close = group['Close'].values
        for i in range(1, len(close)):
            if close[i] > close[i-1]:
                run[i] = run[i-1]+1 if run[i-1]>=0 else 1
            elif close[i] < close[i-1]:
                run[i] = run[i-1]-1 if run[i-1]<=0 else -1
            else:
                run[i] = 0
        group['Trend_Run'] = run
        df_list.append(group)
    return pd.concat(df_list, ignore_index=True)

df = detect_trends(df)
df.head()


Price,Date,Open,High,Low,Close,Volume,Ticker,Sector,SMA_5,SMA_20,Daily_Return,Trend_Run
0,2023-01-03,128.468194,129.079567,122.443165,123.330650,112117500,AAPL,Technology,NaN,NaN,NaN,0
1,2023-01-04,125.125343,126.870731,123.340517,124.602715,89113600,AAPL,Technology,NaN,NaN,0.010314,1
2,2023-01-05,125.361991,125.993089,123.024955,123.281334,80962700,AAPL,Technology,NaN,NaN,-0.010605,-1
3,2023-01-06,124.257586,128.478056,123.153159,127.817375,87754700,AAPL,Technology,NaN,NaN,0.036794,1
4,2023-01-09,128.655553,131.554669,128.083618,128.339996,70790800,AAPL,Technology,125.474414,NaN,0.004089,2


Visualization Checking

In [10]:
# Pick a single ticker for visualization
ticker_to_plot = 'AAPL'
plot_df = df[df['Ticker'] == ticker_to_plot]

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df['Date'], y=plot_df['Close'],
                         mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=plot_df['Date'], y=plot_df['SMA_5'],
                         mode='lines', name='SMA 5'))
fig.add_trace(go.Scatter(x=plot_df['Date'], y=plot_df['SMA_20'],
                         mode='lines', name='SMA 20'))

fig.update_layout(title=f'{ticker_to_plot} Price with SMAs',
                  xaxis_title='Date',
                  yaxis_title='Price (USD)')
fig.show()


In [11]:
fig = px.line(df[df['Ticker'] == 'AAPL'], x='Date', y='Daily_Return',
              title='Daily Returns of AAPL')
fig.show()

In [12]:
fig = px.line(df[df['Ticker'] == 'AAPL'], x='Date', y='Trend_Run',
              title='Trend Runs of AAPL')
fig.show()